In [567]:
from pynq import Overlay
import numpy as np
import random
from pynq import allocate
from scipy.io import wavfile

overlay = Overlay("/home/xilinx/pynq/overlays/maxVal/maxValue2.bit")
SAMPLES = 1024

In [575]:
# FILE = "E7_chord_trimmed.wav"
FILE = "E_string_sample.wav"

def read_wav(filename):
    return wavfile.read(filename)

def do_fft(signal, sample_rate):
    fft_spectrum = np.fft.rfft(signal)
    freq = np.fft.rfftfreq(signal.size, d=1./sample_rate)  # get frequency values of FFT
    fft_spectrum_abs = np.abs(fft_spectrum)  # compute absolute value of FFT plot
    return freq, fft_spectrum_abs

sample_rate, data = read_wav(FILE)
freq, fft = do_fft(data[22050:66150,0], sample_rate) # only use one channel, 1 second of data
buffer = fft[:SAMPLES]

In [576]:
max_ip = overlay.max_DMA.get_top_freq_0
dma = overlay.max_DMA.axi_dma_0
input_buffer = allocate(shape=(SAMPLES,), dtype=np.csingle)

In [577]:
input_buffer[:] = buffer
print(input_buffer)

[4.7439355e+09+0.j 5.2450022e+09+0.j 5.8148101e+09+0.j ...
 2.2777695e+09+0.j 2.6484434e+09+0.j 2.1551035e+09+0.j]


In [578]:
def run_kernel():
    dma.sendchannel.transfer(input_buffer)
    dma.sendchannel.wait()

In [579]:
run_kernel()

In [580]:
max_ip.write(0x00, 0x1)
max_ip.read(0x10)

1329

In [581]:
print(np.argmax(input_buffer))

305


In [582]:
max_ip.register_map

RegisterMap {
  top_freq = Register(top_freq=1329),
  top_freq_ctrl = Register(top_freq_ap_vld=1, RESERVED=0)
}